In [2]:
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Wczytanie danych
data = pd.read_csv("raw_data/arrhythmia/arrhythmia.data", header=None, na_values='?')

# Przypisanie ostatniej kolumny jako target
data.rename(columns={data.columns[-1]: 'target'}, inplace=True)

# Konwersja klas na binarne:
# 0 (Normalny), 1 (wszystkie inne)
data['target'] = data['target'].apply(lambda x: 0 if x == 1 else 1)

# Tworzenie katalogu 'data', jeśli nie istnieje
os.makedirs('data', exist_ok=True)

# Zapisanie pełnych danych do CSV
data.to_csv('data/arrhythmia.csv', index=False)
print("Dane zostały zapisane w pliku 'data/arrhythmia.csv'.")


Dane zostały zapisane w pliku 'data/arrhythmia.csv'.


In [1]:
import pandas as pd
import numpy as np
import os
from scipy.sparse import csr_matrix

# Funkcja do wczytywania danych Dexter (sparse format)
def load_dexter_data_sparse(filepath_data, filepath_labels, num_features=20000):
    rows, cols, vals = [], [], []
    with open(filepath_data, 'r') as file:
        for row_idx, line in enumerate(file):
            elements = line.strip().split()
            for el in elements:
                col_idx, val = el.split(":")
                rows.append(row_idx)
                cols.append(int(col_idx)-1)  # indeksy zaczynają się od 1
                vals.append(int(val))

    X_sparse = csr_matrix((vals, (rows, cols)), shape=(row_idx+1, num_features))
    X = pd.DataFrame(X_sparse.toarray())

    y = pd.read_csv(filepath_labels, header=None, names=['target'])
    
    # Połączenie cech i etykiet w jedną tabelę
    df = X.copy()
    df['target'] = y['target']
    return df

# Ścieżki do plików
train_data_path = 'raw_data/dexter/DEXTER/dexter_train.data'
train_labels_path = 'raw_data/dexter/DEXTER/dexter_train.labels'

# Wczytanie danych
dexter_df = load_dexter_data_sparse(train_data_path, train_labels_path)

# Wyświetlenie pierwszych 5 wierszy danych treningowych
print("Tabela danych treningowych (pierwsze 5 wierszy):")
print(dexter_df.head())

# Podstawowe statystyki dla danych treningowych
print("\nPodstawowe statystyki danych treningowych:")
print(dexter_df.describe())

# Liczba unikalnych klas
print("\nUnikalne klasy w danych treningowych:")
print(dexter_df['target'].unique())

# Tworzenie katalogu 'data', jeśli nie istnieje
os.makedirs('data', exist_ok=True)

# Zapis danych do CSV
dexter_df.to_csv('data/dexter.csv', index=False)
print("Dane zostały zapisane w pliku 'data/dexter.csv'.")

Tabela danych treningowych (pierwsze 5 wierszy):
   0  1  2  3  4  5  6  7  8    9  ...  19991  19992  19993  19994  19995  \
0  0  0  0  0  0  0  0  0  0  105  ...      0      0      0      0      0   
1  0  0  0  0  0  0  0  0  0    0  ...      0      0      0      0      0   
2  0  0  0  0  0  0  0  0  0    0  ...      0      0      0      0      0   
3  0  0  0  0  0  0  0  0  0    0  ...      0      0      0      0      0   
4  0  0  0  0  0  0  0  0  0    0  ...      0      0      0      0      0   

   19996  19997  19998  19999  target  
0      0      0     56      0       1  
1      0      0      0      0      -1  
2      0      0      0      0       1  
3      0      0      0      0      -1  
4      0      0      0      0       1  

[5 rows x 20001 columns]

Podstawowe statystyki danych treningowych:
           0      1      2           3      4           5      6           7  \
count  300.0  300.0  300.0  300.000000  300.0  300.000000  300.0  300.000000   
mean     0.0    0.

In [ ]:
# Arrhythmia

import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Funkcja do wczytania i przygotowania danych
def load_and_prepare_data(filepath):
    data = pd.read_csv(filepath)
    
    # Podział na cechy i target
    X = data.drop(columns=['target'])
    y = data['target']
    
    # Uzupełnianie brakujących wartości średnią
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    
    # Standaryzacja danych
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    
    return X_scaled, y

# Ścieżki do plików
arrhythmia_path = 'data/arrhythmia.csv'
dexter_path = 'data/dexter.csv'

# Wczytanie i przygotowanie danych
X_arrhythmia, y_arrhythmia = load_and_prepare_data(arrhythmia_path)
X_dexter, y_dexter = load_and_prepare_data(dexter_path)

# Funkcja do trenowania i oceny regresji logistycznej
def train_and_evaluate(X, y, dataset_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    
    # Regresja logistyczna bez regularyzacji
    model_no_reg = LogisticRegression(penalty=None, max_iter=1000)
    model_no_reg.fit(X_train, y_train)
    y_pred_no_reg = model_no_reg.predict(X_test)
    
    print(f"\n=== Wyniki dla {dataset_name} - bez regularyzacji ===")
    print(classification_report(y_test, y_pred_no_reg))
    print("Macierz pomyłek:")
    print(confusion_matrix(y_test, y_pred_no_reg))
    
    # Regresja logistyczna z regularyzacją L2 (GridSearchCV dla parametru C)
    param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    grid = GridSearchCV(LogisticRegression(max_iter=1000, penalty='l2'), param_grid, cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred_reg = best_model.predict(X_test)
    
    print(f"\n=== Wyniki dla {dataset_name} - z regularyzacją (L2) ===")
    print("Najlepszy parametr C:", grid.best_params_['C'])
    print(classification_report(y_test, y_pred_reg))
    print("Macierz pomyłek:")
    print(confusion_matrix(y_test, y_pred_reg))

# Trening i ocena dla obu zbiorów danych
train_and_evaluate(X_arrhythmia, y_arrhythmia, "Arrhythmia")
train_and_evaluate(X_dexter, y_dexter, "Dexter")



=== Wyniki dla Arrhythmia - bez regularyzacji ===
              precision    recall  f1-score   support

           0       0.76      0.72      0.74        61
           1       0.69      0.73      0.71        52

    accuracy                           0.73       113
   macro avg       0.72      0.73      0.72       113
weighted avg       0.73      0.73      0.73       113

Macierz pomyłek:
[[44 17]
 [14 38]]

=== Wyniki dla Arrhythmia - z regularyzacją (L2) ===
Najlepszy parametr C: 0.1
              precision    recall  f1-score   support

           0       0.81      0.85      0.83        61
           1       0.82      0.77      0.79        52

    accuracy                           0.81       113
   macro avg       0.81      0.81      0.81       113
weighted avg       0.81      0.81      0.81       113

Macierz pomyłek:
[[52  9]
 [12 40]]

=== Wyniki dla Dexter - bez regularyzacji ===
              precision    recall  f1-score   support

          -1       0.97      0.87      0.